# sub sample by density

In [1]:
import pickle
import load
import numpy as np
import matplotlib.pyplot as plt
from tree import halomodule as hmo

In [4]:
wdir = %pwd
wdir = wdir + '/'
print(wdir)

/media/hoseung/btrfs/Horizon-AGN/


In [5]:
gcat = hmo.Halo(nout=782, is_gal=True, base=wdir)

[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =/media/hoseung/btrfs/Horizon-AGN/


In [6]:
x = gcat.data["x"]
y = gcat.data["y"]
z = gcat.data["z"]

In [7]:
x_ind = np.argsort(x)
y_ind = np.argsort(y)
z_ind = np.argsort(z)

In [52]:
d5=[]
N5=[]

Ncut = 50000

for i, gal in enumerate(gcat.data):
    xg,yg,zg = gal["x"],gal["y"],gal["z"]
    x_ok = np.abs(x_ind - xg) < Ncut
    y_ok = np.abs(y_ind - yg) < Ncut
    z_ok = np.abs(z_ind - zg) < Ncut
    #all_ok = np.abs(z_ind[xy_ok] - zg) < 5000
    
    all_ok = np.where(x_ok * y_ok * z_ok)[0]
    
    dist = np.sqrt(np.square(xg - x[all_ok]) + 
                   np.square(yg - y[all_ok]) + 
                   np.square(zg - z[all_ok]))
    
    d5.append(sorted(dist)[5] * gcat.info.pboxsize) # 0 = itself.
    N5.append(sum(dist < 5/gcat.info.pboxsize))
    #print(len(all_ok), d5, N5)

In [96]:
d5_90 = d5[np.argsort(d5)[np.ceil(0.95 * len(d5)).astype(int)]]
i_isol = np.where(d5 >= d5_90)[0]

d5_10 = d5[np.argsort(d5)[np.ceil(0.05 * len(d5)).astype(int)]]
i_dense = np.where(d5 <= d5_10)[0]

In [101]:
fig, ax = plt.subplots(1,2)
ax[0].hist2d(gcat.data["x"], gcat.data["y"], bins=400)
ax[1].hist2d(gcat.data["x"], gcat.data["y"], bins=400)
ax[0].scatter(gcat.data["x"][i_dense], gcat.data["y"][i_dense])

plt.show()

In [11]:
ids = gcat.data["id"][i_dense]
fatherID = pickle.load(open(wdir + "fatherID.pickle", "rb"))
fatherIDx = pickle.load(open(wdir + "fatherIDx.pickle", "rb"))
fatherMass = pickle.load(open(wdir + "fatherMass.pickle", "rb"))
tt = pickle.load(open(wdir + "Tree.pickle", "rb"))

t_now = tt[tt['zred'] == tt["zred"][-1]] # zred = 0.017xx?

final_gals = [t_now['idx'][gid-1] for gid in ids] # Every final galaxy has (short or long) tree.

np.savetxt(wdir + "top5_dense_IDs.txt", (ids, final_gals), fmt='%10d')

In [13]:
ids, final_gals = np.genfromtxt(wdir + "top5_dense_IDs.txt", dtype=int)

In [15]:
import numpy.lib.recfunctions as rcfuncs
import tree.hmutils as hmu

allgals=[]
for final_idx in final_gals:
    atree = hmu.extract_main_tree(tt, final_idx, fatherIDx, fatherMass)
    allgals.append(rcfuncs.append_fields(atree,
                                         "tree_root_id",
                                         data=np.full(len(atree),final_idx, dtype=np.int64),
                                         usemask=False))
    # Add main progenitor tag



In [17]:
pickle.dump(allgals, open(wdir +"All_dense_tree.pickle", "wb"))